In [4]:
import sys
sys.path.append('..')
from CompoundPoisson2 import CompoundPoisson2
from MCABS import MCLossAbs
import numpy as np
import scipy.stats as st
import math
from scipy.optimize import minimize
import time

In [5]:
#class for loss function of the following form :  l(x) = 1 / (1 + alpha)(sum(exp(beta * xi)) + alpha*exp(beta *sum(xi)))- (alpha+d)/(alpha+1)
class MCLossFunction2(MCLossAbs):
    def __init__(self, X, alpha):
        self.__alpha = alpha
        super(MCLossFunction2, self).__init__(X)
        
    def shortfall_risk(self, m = None):
        m = self._check_argument(m)
        #This substract from the col i of X, m_i
        #X is a matrix of dim columns and N rows where N is the sample's length
        x_minus_m = np.subtract(self.X, m)
        #axis = 1 means we sum the elements of each row
        mean_sum = np.mean(np.sum(x_minus_m, axis = 1))
        pos_part = np.maximum(x_minus_m, 0)
        mean_sum_2 = np.mean(np.sum(pos_part ** 2, axis = 1))
        cross_term = 0.
        for i in range(self.dim):
            for j in range(i + 1, self.dim):
                cross_term += np.mean(np.multiply(pos_part[:, i], pos_part[:, j]))
        return mean_sum + 0.5 * mean_sum_2 + self.__alpha * cross_term
    
    def shortfall_risk_jac(self, m):
        m = self._check_argument(m)
        x_minus_m = np.subtract(self.X, m)
        pos_part = np.maximum(x_minus_m, 0) 
        pos_part_mean = np.mean(pos_part, axis = 0)
        cross_jac = []
        for i in range(self.dim):
            temp = 0.
            indic_i = np.sign(pos_part[:, i])
            for k in range(self.dim):
                if k != i:
                    temp += self.__alpha * np.mean(np.multiply(pos_part[:, k], indic_i))
            cross_jac.append(temp)
        return pos_part_mean + 1. + cross_jac

In [ ]:
#Case: rho = 0, alpha = 1, beta = 1

#M is the length of the sample
np.random.seed(seed=0)
M = 500000
corr = 0.9
mu = [0., 0.]
sigma = [[1., corr], [corr, 1.]]

rv = st.multivariate_normal(mean = mu, cov = sigma, allow_singular = True)
distr = rv.rvs(size = M)

alpha, beta = 1., 1.

maxiter = 3500

start_time = time.time()

loss = MCLossFunction2(distr, alpha)


init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})


res = minimize(loss.objective, init, 
              jac = loss.objective_jac, 
              constraints = cons, 
               method='SLSQP',
               options={'maxiter': maxiter, 'disp': True})
print(res)
print("--- MC Method: %s seconds ---" % (time.time() - start_time))